# Explore here

In [215]:
# Your code here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB,GaussianNB,BernoulliNB
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score


In [205]:
df = pd.read_csv('/workspaces/pauline-Proyecto-Naive-Bayes/data/raw/playstore_reviews.csv')
#Como dicho en las instrucciones, eliminamos package_name, ya que de eso no depende la clasificación de la reseña
df.drop(['package_name'], axis=1, inplace=True)
df.head()

,review,polarity
0,privacy at least put some option appear offli...,0
1,"messenger issues ever since the last update, ...",0
2,profile any time my wife or anybody has more ...,0
3,the new features suck for those of us who don...,0
4,forced reload on uploading pic on replying co...,0


In [206]:
#Quiero saber cuantas filas hay :
print(df.shape)
#Tipo de data y check si hay nulos/faltantes
print(df.info())
# Revisamos que no haya fila duplicada:
sum(df.duplicated())

(891, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   review    891 non-null    object
 1   polarity  891 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 14.0+ KB
None


0

In [207]:
#Eliminar espacios y convertir a minúsculas el texto: 
df["review"] = df["review"].str.strip().str.lower()

#Y dividimos los datos en conjuntos de entrenamiento y testeo
X = df['review']
y = df[['polarity']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [208]:
X_train.head()

331    just did the latest update on viber and yet ag...
733    keeps crashing it only works well in extreme d...
382    the fail boat has arrived the 6.0 version is t...
704    superfast, just as i remember it ! opera mini ...
813    installed and immediately deleted this crap i ...
Name: review, dtype: object

In [209]:
vectorizer = CountVectorizer(stop_words = "english") #ese de stop_word se usa para eliminar las palabras más comunes y sin valor informativo
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)



In [210]:
#Chequeamos con algunas palabras o características que han sido extraídas del texto durante el proceso de "vectorización".
vectorizer.get_feature_names_out()


array(['000', '04', '0x', ..., 'žŕ', 'žŕľ', 'ˇŕ'],
      shape=(3310,), dtype=object)

In [211]:
X_test_vec.toarray()[1]

array([0, 0, 0, ..., 0, 0, 0], shape=(3310,))

In [212]:
#Mostrar una review de prueba (texto)
print(f'Review de prueba: {X_test.iloc[1]}')


print('\nPalabras del conjunto de entrenamiento que también aparecen en la review de test junto con su aparición:\n')
for i, cont in enumerate(X_test_vec.toarray()[1]):
  if cont!=0:
    print(f'Palabra: "{vectorizer.get_feature_names_out()[i]}"')
    print(f'--> Aparece {cont} veces en la review.')


Review de prueba: whatsapp i use this app now that blackberry messenger has basically gone away. my friends & family live all over the world. this really helps keep us in touch!

Palabras del conjunto de entrenamiento que también aparecen en la review de test junto con su aparición:

Palabra: "app"
--> Aparece 1 veces en la review.
Palabra: "away"
--> Aparece 1 veces en la review.
Palabra: "basically"
--> Aparece 1 veces en la review.
Palabra: "family"
--> Aparece 1 veces en la review.
Palabra: "friends"
--> Aparece 1 veces en la review.
Palabra: "gone"
--> Aparece 1 veces en la review.
Palabra: "helps"
--> Aparece 1 veces en la review.
Palabra: "live"
--> Aparece 1 veces en la review.
Palabra: "messenger"
--> Aparece 1 veces en la review.
Palabra: "really"
--> Aparece 1 veces en la review.
Palabra: "touch"
--> Aparece 1 veces en la review.
Palabra: "use"
--> Aparece 1 veces en la review.
Palabra: "whatsapp"
--> Aparece 1 veces en la review.
Palabra: "world"
--> Aparece 1 veces en la r

**Paso 3: Construye un naive bayes.** Elegimos **Multinomial** porque variable predictora es categorica, con varias posibilidades.   (Gauss es para regresion y Bernoulli cuando las variables son binarias).

In [218]:
#Inicializar y entrenar el clasificador Naive Bayes Multinomial
clf = MultinomialNB().fit(X_train_vec, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = clf.predict(X_test_vec)


# Evaluar el rendimiento del modelo

print(f'Evaluación del modelo: {classification_report(y_test, y_pred)}\n')
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')






Evaluación del modelo:               precision    recall  f1-score   support

           0       0.84      0.90      0.87       126
           1       0.73      0.60      0.66        53

    accuracy                           0.82       179
   macro avg       0.79      0.75      0.77       179
weighted avg       0.81      0.82      0.81       179


Accuracy: 0.8156424581005587


/home/vscode/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Testeamos los 2 otros
- **GAUSS**:


In [219]:
#probamos con los 2 otros : 

#Inicializar y entrenar el clasificador Naive GaussianNB
clf = GaussianNB().fit(X_train_vec.toarray(), y_train)

# Realizar predicciones en el conjunto de prueba
y_pred_gauss = clf.predict(X_test_vec.toarray())


# Evaluar el rendimiento del modelo

print(f'Evaluación del modelo: {classification_report(y_test, y_pred_gauss)}\n')
print(f'Accuracy: {accuracy_score(y_test, y_pred_gauss)}')


Evaluación del modelo:               precision    recall  f1-score   support

           0       0.85      0.88      0.86       126
           1       0.69      0.62      0.65        53

    accuracy                           0.80       179
   macro avg       0.77      0.75      0.76       179
weighted avg       0.80      0.80      0.80       179


Accuracy: 0.8044692737430168


/home/vscode/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


- BERNOUILLI

In [222]:
#Inicializar y entrenar el clasificador Naive GaussianNB
clf = BernoulliNB
clf.fit(X_train_vec.toarray(), y_train) 

# Realizar predicciones en el conjunto de prueba
y_pred_ber = clf.predict(X_test_vec.toarray())


# Evaluar el rendimiento del modelo

print(f'Evaluación del modelo: {classification_report(y_test, y_pred_ber)}\n')
print(f'Accuracy: {accuracy_score(y_test, y_pred_ber)}')

AttributeError: 'numpy.ndarray' object has no attribute '_validate_params'